⚠️ This project is mandatory for certification bloc #1.

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 


## Company's description 📇

<a href='https://www.kayak.com' target='_blank'>Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href='https://www.bookingholdings.com/' target='_blank'>Booking Holdings</a> which now holds: 

* <a href='https://booking.com/' target='_blank'>Booking.com</a>
* <a href='https://kayak.com/' target='_blank'>Kayak</a>
* <a href='https://www.priceline.com/' target='_blank'>Priceline</a>
* <a href='https://www.agoda.com/' target='_blank'>Agoda</a>
* <a href='https://Rentalcars.com/' target='_blank'>RentalCars</a>
* <a href='https://www.opentable.com/' target='_blank'>OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href='https://one-week-in.com/35-cities-to-visit-in-france/' target='_blank'>One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
['Mont Saint Michel',
'St Malo',
'Bayeux',
'Le Havre',
'Rouen',
'Paris',
'Amiens',
'Lille',
'Strasbourg',
'Chateau du Haut Koenigsbourg',
'Colmar',
'Eguisheim',
'Besancon',
'Dijon',
'Annecy',
'Grenoble',
'Lyon',
'Gorges du Verdon',
'Bormes les Mimosas',
'Cassis',
'Marseille',
'Aix en Provence',
'Avignon',
'Uzes',
'Nimes',
'Aigues Mortes',
'Saintes Maries de la mer',
'Collioure',
'Carcassonne',
'Ariege',
'Toulouse',
'Montauban',
'Biarritz',
'Bayonne',
'La Rochelle']
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

# Let's start

In [ ]:
# Import libs:

import pandas as pd
import requests
import numpy as np
import plotly.express as px
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import os
from bs4 import BeautifulSoup
import time


## Get weather data with an API

In [ ]:
# Import list of cities

cities = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle",
]


In [ ]:
# Creating a dataframe from the list of cities
df_cities = pd.DataFrame(data=cities, columns=["city"])
df_cities = df_cities.reset_index()
df_cities = df_cities.rename(columns={"index": "id"})
df_cities


### Nominatim API : get the gps coordinates of all the cities => https://nominatim.org/release-docs/latest/

In [ ]:
# Try a city with blank space in name to see if formatting needed:

params = {"city": "La Rochelle", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


In [ ]:
# I don't know 'Chateau du Haut Koenigsbourg', check if Nominatim knows it

params = {"city": "Chateau du Haut Koenigsbourg", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r


In [ ]:
# 'Gorges du Verdon' isn't a city but a canyon, check if Nominatim works with or not

params = {"city": "Gorges du Verdon", "country": "France", "format": "json"}

r = requests.get("https://nominatim.openstreetmap.org/search?", params).json()

r

# Doesnt' works... Google search : 'Castellane' will be use for Gorges du Verdon


Ariege is a department and not a city, its prefecture is 'Foix'. We'll use this city

#### Request API

In [ ]:
# Creating a copy of df_cities to store the coordinates from Nominatim API

df_gps = df_cities.copy()
lat_list = []
lon_list = []

for i in cities:
    print(f"Request for city: {i}")
    params = {"city": i, "country": "France", "format": "json"}
    # No 'Gorges du Verdon' city, replacing by 'Castellane'
    if i == "Gorges du Verdon":
        i = "Castellane"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    # 'Ariege' not a city, using the prefecture instead -> 'Foix'
    elif i == "Ariege":
        i = "Foix"
        r = requests.get(
            f"https://nominatim.openstreetmap.org/search?city={i}&country=France&format=json"
        ).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])
    else:
        r = requests.get(f"https://nominatim.openstreetmap.org/search?", params).json()
        lat_list.append(r[0]["lat"])
        lon_list.append(r[0]["lon"])

# Adding the coordinates to the dataframe
df_gps["lat"] = lat_list
df_gps["lon"] = lon_list


In [ ]:
df_gps.head()


In [ ]:
df_gps.info()


### OpenWeather : get weather of the week => https://openweathermap.org

In [ ]:
# try at St Malo
parameters = {
    "lat": 48.649518,
    "lon": -2.0260409,
    "exclude": "current,minutely,hourly",
    "units": "metric",
    "appid": "aa423e6694bf72625fe1fe31544949dc",
    "lang": "fr",
}

r = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall", params=parameters
).json()

r


In [ ]:
# Weather in 3 days at St Malo
r["daily"][3:]


In [ ]:
day3 = r["daily"][3]  # Weather in 3 days

# desciption of the main weather in 3 days at St Malo
day3["weather"][0]["main"]


In [ ]:
df_gps_weather = df_gps.copy(deep=True)
my_api_key = "aa423e6694bf72625fe1fe31544949dc"

temperatures_list = []
rain_list = []
weather_list = []

days = list(range(1, 8))

for i in df_gps_weather.itertuples():
    lat = i.lat
    lon = i.lon

    parameters = {
        "lat": {lat},
        "lon": {lon},
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": "aa423e6694bf72625fe1fe31544949dc",
        "lang": "fr",
    }

    r = requests.get(
        f"https://api.openweathermap.org/data/2.5/onecall?", parameters
    ).json()
    forecast_7days = r["daily"][1:]  # Getting the weather data for the next 7 days
    temperatures = [int(d["feels_like"]["day"]) for d in forecast_7days]
    rain = [int(d["pop"] * 100) for d in forecast_7days]
    weather = [str(d["weather"][0]["main"]) for d in forecast_7days]
    temperatures_list.append(temperatures)
    rain_list.append(rain)
    weather_list.append(weather)

df_gps_weather["jour_+x"] = [days for _ in range(len(df_gps_weather))]
df_gps_weather["temperature_ressentie"] = temperatures_list
df_gps_weather["probabilite_de_pluie"] = rain_list
df_gps_weather["meteo_principale"] = weather_list
# df_weather['rang'] = df_weather['probabilite_de_pluie'].sort_values()
df_gps_weather["score"] = df_gps_weather.apply(
    lambda x: ((np.mean(x["temperature_ressentie"])))
    - (np.mean(x["probabilite_de_pluie"]) / 10),
    axis=1,
).astype(int)


In [ ]:
df_gps_weather


In [ ]:
df_gps_weather.info()


In [ ]:
df_gps_weather[["lat", "lon"]] = df_gps_weather[["lat", "lon"]].astype(float)


### List of cities where the weather will be the nicest within the next 7 days

In [ ]:
df_gps_weather.sort_values(by=["score"], ascending=False)


### Save all the results in a `.csv` file

In [ ]:
df_gps_weather.to_csv("df_gps_weather.csv", index=False)


### Plotly to display the best destinations on a map

In [ ]:
df_plotly = df_gps_weather.apply(
    pd.Series.explode
)  # To obtain a line per day and per city

df_plotly[["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]] = df_plotly[
    ["jour_+x", "temperature_ressentie", "probabilite_de_pluie"]
].astype(int)

fig = px.scatter_mapbox(
    df_plotly,
    lat="lat",
    lon="lon",
    hover_name="city",
    zoom=4,
    hover_data=["meteo_principale", "probabilite_de_pluie", "temperature_ressentie"],
    color="temperature_ressentie",
    color_continuous_scale="thermal",
    mapbox_style="carto-positron",
)
fig.show()


## Scrape Booking.com 

### Get hotels and their URLs

In [ ]:
class BookingSpider(scrapy.Spider):
    # Name of my spider
    name = "booking_spider"
    cities = df_gps_weather.city
    # Url to start my spider from
    start_urls = [
        "https://www.booking.com/index.fr.html",
    ]

    # Callback function that will be called when starting my spider
    def parse(self, response):
        for x in cities:
            yield scrapy.FormRequest.from_response(
                response, formdata={"ss": x}, callback=self.after_search
            )

    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]
        booking = response.css(".d4924c9e74")

        for data in booking:

            yield {
                "ville": cities,
                "hotels": data.css("a div.fcab3ed991.a23c043802::text").getall(),
                "liens": data.css("h3.a4225678b2 a::attr(href)").getall(),
            }

        try:
            next_page = response.css("a.paging-next").attrib["href"]
        except KeyError:
            logging.info("No next page. Terminating crawling process.")
        else:
            yield response.follow(next_page, callback=self.after_search)


# Name of the file where the results will be saved
filename = "hotels.json"

# If file already exists, delete before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir():
    os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(
    settings={
        "USER_AGENT": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1",
        "LOG_LEVEL": logging.INFO,
        "FEEDS": {
            filename: {"format": "json"},
        },
        "AUTOTHROTTLE_ENABLED": True,
    }
)

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()


### Get hotels' coordinates

In [ ]:
# Create a DataFrame from the json file

df = pd.read_json("hotels.json")
df.head()


In [ ]:
# Reduce number of hotels per city to 20 as requested by the task
for i in range(len(df["ville"])):
    df["hotels"][i] = df["hotels"][i][0:20]
    df["liens"][i] = df["liens"][i][0:20]


In [ ]:
# BeautifulSoup iterates on each URL to get the hotel's score, GPS coordinates and description.
df["lat"] = 0
df["lon"] = 0
df["description"] = "---"
df["score"] = 0.0

navigator = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)"

for i in range(len(df["liens"])):
    lat_list = []
    lon_list = []
    description_list = []
    score_list = []

    hotel_list = df["liens"][i]

    for i2 in hotel_list:

        # Sometimes BeautifulSoup doesn't manage to gather data. When it fails, it tries again.
        try:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")
        except:
            page = requests.get(i2, headers={"User-Agent": navigator})
            soup = BeautifulSoup(page.text, "html.parser")

        lat_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[0]
        )
        lon_list.append(
            soup.select("p.address.address_clean a")[0]
            .get("data-atlas-latlng")
            .split(",")[1]
        )
        description_list.append(
            soup.select("div#property_description_content")[0].get_text()
        )

        try:
            score_list.append(soup.select("div.b5cd09854e.d10a6220b4")[0].get_text())

        except:
            # 2 hotels over the 700 that I am going to scrap dont have a score yet but I still need one for the visualization. I set it to 1.
            score_list.append("1.0")

        time.sleep(1.4)

    df["lat"][i] = lat_list
    df["lon"][i] = lon_list
    df["description"][i] = description_list
    df["score"][i] = score_list

    print(f"city {df['ville'].iloc[i]} done")


In [ ]:
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df


In [ ]:
df["ville"] = df["ville"].str.replace("+", " ")


In [ ]:
df.to_csv("df_gps_hotels.csv", index=False)


In [ ]:
df_test = pd.read_csv("df_gps_hotels.csv")
df_test.head()


In [ ]:
df_test2 = pd.read_csv("df_gps_weather.csv")
df_test2.head()


## Create your data lake using S3

## ETL